In [10]:
inference_size = 4096

In [11]:
import sys
import torch
from models import create_model
from options.test_options import TestOptions

# Path to your pre-trained weights
weights_path = "/Users/yahyarahhawi/Downloads/65_net_G_A.pth"

# Override sys.argv to inject minimal CLI args for TestOptions
original_argv = sys.argv
sys.argv = [
    original_argv[0],
    '--dataroot', 'dummy',        # Needed to satisfy required '--dataroot' argument
    '--model', 'cycle_gan',       # Specifies the CycleGAN model
    '--dataset_mode', 'single',   # We are feeding images manually
    '--gpu_ids', '-1',            # Disable GPU explicitly to avoid CUDA dependency
]

# Parse the options
opt = TestOptions().parse()

# Restore sys.argv to its original state
sys.argv = original_argv

# Set additional options manually
opt.isTrain = False  # Indicates we're running in evaluation/test mode
opt.no_dropout = True  # No dropout for inference
opt.batch_size = 1  # Inference only works with batch size = 1
opt.load_size = inference_size  # Resize to 256x256 during preprocessing (adjust as needed)
opt.crop_size = inference_size  # Crop size must match load size for this example

# Set the device to MPS if available, otherwise CPU
device = torch.device("mps") if torch.backends.mps.is_available() else torch.device("cpu")
print(f"Using device: {device}")

# Create the CycleGAN model
cycleGAN = create_model(opt)
cycleGAN.eval()  # Set the model to evaluation mode

# Manually load netG_A weights
state_dict = torch.load(weights_path, map_location=device)
cycleGAN.netG_A.load_state_dict(state_dict)

# Move the model to the device
cycleGAN.netG_A = cycleGAN.netG_A.to(device)

# Access the generator (netG_A)
model_netG_A = cycleGAN.netG_A

# Print a summary to verify everything is working
print("CycleGAN model created successfully!")
print(f"Generator loaded: {model_netG_A}")

----------------- Options ---------------
             aspect_ratio: 1.0                           
               batch_size: 1                             
          checkpoints_dir: ./checkpoints                 
                crop_size: 256                           
                 dataroot: dummy                         	[default: None]
             dataset_mode: single                        	[default: unaligned]
                direction: AtoB                          
          display_winsize: 256                           
                    epoch: latest                        
                     eval: False                         
                  gpu_ids: -1                            	[default: 0]
                init_gain: 0.02                          
                init_type: normal                        
                 input_nc: 3                             
                  isTrain: False                         	[default: None]
                load_i

/var/folders/p3/zqj4hsr94qs443gkb32p5h_00000gn/T/ipykernel_90603/1165555012.py:41: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load(weights_path, map_lo

CycleGAN model created successfully!
Generator loaded: ResnetGenerator(
  (model): Sequential(
    (0): ReflectionPad2d((3, 3, 3, 3))
    (1): Conv2d(3, 64, kernel_size=(7, 7), stride=(1, 1))
    (2): InstanceNorm2d(64, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
    (3): ReLU(inplace=True)
    (4): Conv2d(64, 128, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (5): InstanceNorm2d(128, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 256, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (8): InstanceNorm2d(256, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
    (9): ReLU(inplace=True)
    (10): ResnetBlock(
      (conv_block): Sequential(
        (0): ReflectionPad2d((1, 1, 1, 1))
        (1): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1))
        (2): InstanceNorm2d(256, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
        (3): ReLU(inplace

In [ ]:
# Create a dummy input with the same shape as your input images
dummy_input = torch.randn(1, 3, inference_size, inference_size).to(device)

# Trace the model to produce a TorchScript version
traced_model = torch.jit.trace(model_netG_A, dummy_input)

# Save the TorchScript model (optional, for debugging)
traced_model_path = "/Users/yahyarahhawi/Developer/Film/cycleGAN_traced.pt"
traced_model.save(traced_model_path)

print(f"TorchScript model saved to {traced_model_path}")

In [ ]:
import coremltools as ct

# Convert the TorchScript model to Core ML
coreml_model = ct.convert(
    traced_model,
    inputs=[ct.ImageType(name="input", shape=(1, 3, inference_size, inference_size))],
    minimum_deployment_target=ct.target.iOS13  # Specify deployment target if needed
)

# Save the Core ML model with the correct extension
coreml_model_path = "/Users/yahyarahhawi/Developer/Film/cycleGAN.mlpackage"
coreml_model.save(coreml_model_path)

print(f"Core ML model saved to {coreml_model_path}")

Torch version 2.5.1 has not been tested with coremltools. You may run into unexpected errors. Torch 2.4.0 is the most recent version that has been tested.
Running MIL default pipeline:   0%|          | 0/87 [00:00<?, ? passes/s]/Users/yahyarahhawi/miniforge3/envs/detectron2/lib/python3.12/site-packages/coremltools/converters/mil/mil/passes/defs/preprocess.py:273: UserWarning: Output, '309', of the source model, has been renamed to 'var_309' in the Core ML model.
  warnings.warn(msg.format(var.name, new_name))
Translating MIL ==> NeuralNetwork Ops: 100%|██████████| 328/328 [00:01<00:00, 236.54 ops/s]


Core ML model saved to /Users/yahyarahhawi/Developer/Film/cycleGAN.mlpackage


*Test Core ML*

In [ ]:
import coremltools as ct
from PIL import Image
import numpy as np

# Load the Core ML model
coreml_model_path = "/Users/yahyarahhawi/Developer/Film/cycleGAN.mlpackage"
model = ct.models.MLModel(coreml_model_path)

# Load an input image
input_image_path = "/Users/yahyarahhawi/Downloads/epoch017_real_A (1).png"
image = Image.open(input_image_path).convert("RGB")
image = image.resize((inference_size, inference_size))  

# Perform inference
output = model.predict({"input": image})


output_image_data = output["var_309"]  


output_image_data = (output_image_data.squeeze().transpose(1, 2, 0) * 255).clip(0, 255).astype(np.uint8)  # (H, W, C)
output_image = Image.fromarray(output_image_data)

# Save or display the output image
output_image.save("/Users/yahyarahhawi/Developer/Film/output_image.jpg")
output_image.show()